<a href="https://colab.research.google.com/github/akitim/SkillFactory_projects/blob/main/Project_2_EDA_Timofey_Akimkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Загрузка библиотек

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
 
pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок
 
import sys
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
sys.path.clear
sys.path.append('/content/gdrive/MyDrive/SkillFactory')

Mounted at /content/gdrive


#Считывание датафрейма из файла

In [7]:
cacao = pd.read_csv('/content/gdrive/MyDrive/SkillFactory/stud_math.csv')
display(cacao.sample(2))

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
284,GP,F,17,U,GT3,T,2.0,2.0,other,other,course,mother,1.0,2.0,0.0,no,yes,no,no,no,-6.0,yes,yes,no,5.0,4.0,5.0,5.0,4.0,55.0
316,GP,F,18,U,GT3,T,2.0,1.0,services,NaN,course,mother,2.0,2.0,0.0,no,yes,yes,yes,yes,-6.0,yes,yes,no,5.0,3.0,3.0,1.0,0.0,0.0


#Предобработка

In [ ]:
def hasletornum(s):
    ls = [ch for ch in s]
    return any(list(map(lambda x: x.isalnum(), ls)))
